In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
#indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

# pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
# pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

pairs_train = [x for x in pairs if int(x[0].split('_')[-1]) <= 9]
pairs_test = [x for x in pairs if int(x[0].split('_')[-1]) >= 10]

In [7]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [8]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

2024-05-22 07:07:27.968308: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-22 07:07:27.968357: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-22 07:07:27.968376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3035.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-22 07:07:27.968579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
1/1 - 4s - loss: 3.1853 - val_loss: 3.1523 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 3.1355 - val_loss: 3.1253 - 155ms/epoch - 155ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1051 - val_loss: 3.1015 - 183ms/epoch - 183ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.0807 - val_loss: 3.0812 - 158ms/epoch - 158ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.0590 - val_loss: 3.0637 - 160ms/epoch - 160ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0404 - val_loss: 3.0481 - 146ms/epoch - 146ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.0248 - val_loss: 3.0354 - 139ms/epoch - 139ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.0119 - val_loss: 3.0251 - 150ms/epoch - 150ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0010 - val_loss: 3.0164 - 156ms/epoch - 156ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.9915 - val_loss: 3.0081 - 166ms/epoch - 166ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.9822 - val_loss: 2.9999 - 153ms/epoch - 153ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.9730 - val_loss: 2.9920 - 146ms/epoch - 146ms/step
Epoch 1

Epoch 100/2000
1/1 - 0s - loss: 2.6294 - val_loss: 2.6625 - 83ms/epoch - 83ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.6264 - val_loss: 2.6590 - 82ms/epoch - 82ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.6237 - val_loss: 2.6589 - 85ms/epoch - 85ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.6217 - val_loss: 2.6551 - 81ms/epoch - 81ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.6202 - val_loss: 2.6541 - 100ms/epoch - 100ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.6167 - val_loss: 2.6474 - 87ms/epoch - 87ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.6118 - val_loss: 2.6439 - 84ms/epoch - 84ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.6074 - val_loss: 2.6438 - 84ms/epoch - 84ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.6062 - val_loss: 2.6407 - 83ms/epoch - 83ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.6045 - val_loss: 2.6372 - 83ms/epoch - 83ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.5994 - val_loss: 2.6327 - 81ms/epoch - 81ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.5953 - val_loss: 2.6305 - 83ms/epoch - 83ms/step
Ep

Epoch 199/2000
1/1 - 0s - loss: 2.3660 - val_loss: 2.4139 - 83ms/epoch - 83ms/step
Epoch 200/2000
1/1 - 0s - loss: 2.3633 - val_loss: 2.4145 - 75ms/epoch - 75ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.3625 - val_loss: 2.4145 - 77ms/epoch - 77ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.3671 - val_loss: 2.4329 - 76ms/epoch - 76ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.3777 - val_loss: 2.4325 - 76ms/epoch - 76ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.3853 - val_loss: 2.4368 - 97ms/epoch - 97ms/step


In [9]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [10]:
(np.mean(acc), np.mean(prob))

(0.0, 0.0171412)

In [11]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 4s - loss: 3.8087 - val_loss: 3.2956 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2599 - val_loss: 3.1145 - 306ms/epoch - 306ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.0763 - val_loss: 3.0461 - 282ms/epoch - 282ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.0114 - val_loss: 2.9794 - 286ms/epoch - 286ms/step
Epoch 5/2000
1/1 - 0s - loss: 2.9468 - val_loss: 2.9573 - 264ms/epoch - 264ms/step
Epoch 6/2000
1/1 - 0s - loss: 2.9252 - val_loss: 2.9383 - 272ms/epoch - 272ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.9075 - val_loss: 2.9163 - 275ms/epoch - 275ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.8870 - val_loss: 2.9022 - 274ms/epoch - 274ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.8732 - val_loss: 2.8965 - 254ms/epoch - 254ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.8666 - val_loss: 2.8921 - 247ms/epoch - 247ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.8602 - val_loss: 2.8821 - 265ms/epoch - 265ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.8474 - val_loss: 2.8655 - 277ms/epoch - 277ms/step
Epoch 1

In [12]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [13]:
(np.mean(acc), np.mean(prob))

(0.0, 0.0019487024)

In [14]:
x_masked_test

array([[10,  0,  1, 10],
       [10,  0,  2, 10],
       [10,  0,  3, 10],
       ...,
       [19, 18, 15, 19],
       [19, 18, 16, 19],
       [19, 18, 17, 19]])